In [1]:
import time
import numpy as np

# 랜덤 데이터 생성
np.random.seed(42)
x = np.random.randn(100)

# 실제값: a = 2, b = 0
y = 2*x + 0.1*np.random.randn(100) 

# RMSE 로스 펑션 정의
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

In [2]:
x[:10]

array([ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
       -0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004])

In [3]:
y[:10]

array([ 0.85189123, -0.31859313,  1.26110562,  2.96583199, -0.48443532,
       -0.42786883,  3.34704422,  1.55232724, -0.91319373,  1.0776755 ])

## Brute-force 방법 (무작위 대입법)

In [4]:
start_time = time.time()

# -10~10 사이의 값을 1000개의 interval로 쪼갬
a_values = np.linspace(-10, 10, 1000)
b_values = np.linspace(-10, 10, 1000)

min_loss = float("inf")
best_a, best_b = None, None

# a와 b 값들을 iteration
for a in a_values:
    for b in b_values:

        # broadcasting이 일어남 -> a는 scalar 값이지만, x는 (100,) 값을 갖고 있음
        y_pred = a * x + b
        loss = rmse(y, y_pred)

        # 최소 로스를 기록해서, 그 보다 낮은 값이 등장할 경우 로스와 최적의 파라미터를 update
        if loss < min_loss:
            min_loss = loss
            best_a, best_b = a, b


# 소요 시간 확인
brute_force_time = time.time() - start_time
print(
    f"Brute-force method: a = {best_a}, b = {best_b}, RMSE = {min_loss}, Time = {brute_force_time} seconds"
)

Brute-force method: a = 1.9919919919919913, b = 0.010010010010010006, RMSE = 0.09456779060230674, Time = 4.833376884460449 seconds


## Stochastic Gradient Descent

In [5]:
start_time = time.time()

# 무작위의 a,b를 생성
a, b = np.random.randn(), np.random.randn()

# 임의의 학습률
learning_rate = 0.1

# 1000번의 epoch에 걸쳐 업데이트 진행
for epoch in range(1000):
    # 랜덤 샘플 1개를 사용
    random_idx = np.random.choice(len(x))  # SGD uses one random sample per step
    xi, yi = x[random_idx], y[random_idx]

    # 예측값 생성
    y_pred = a * xi + b

    # MSE의 도함수 (RMSE와 최적값이 동일)
    gradient_a = -2 * xi * (yi - y_pred)
    gradient_b = -2 * (yi - y_pred)

    a = a - learning_rate * gradient_a
    b = b - learning_rate * gradient_b

    # 각 100번째 스텝마다 parameter와 loss 값 계산
    if epoch % 100 == 0:
        y_pred = a * x + b
        loss = rmse(y, y_pred)
        print(f"Epoch {epoch}, a = {a}, b = {b}, RMSE = {loss}")

# 소요 시간 확인
sgd_time = time.time() - start_time
print(f"SGD method: a = {a}, b = {b}, RMSE = {loss}, Time = {sgd_time} seconds")

Epoch 0, a = 0.3596477643863195, b = 0.4229484842070944, RMSE = 1.5865199401612435
Epoch 100, a = 1.9419895343642082, b = 0.0655332076753822, RMSE = 0.12329138130155666
Epoch 200, a = 2.01514870339355, b = 0.012580090713466448, RMSE = 0.0980953529378534
Epoch 300, a = 1.9456542858405708, b = -0.019332105434106503, RMSE = 0.10196803408445895
Epoch 400, a = 2.0164010405874455, b = -0.01571578726957652, RMSE = 0.0999666504919011
Epoch 500, a = 2.0401515775828365, b = 0.008233485799572428, RMSE = 0.10612700340871035
Epoch 600, a = 1.9708466358825485, b = 0.016315602777042137, RMSE = 0.09648153419077189
Epoch 700, a = 2.0131361553083407, b = 0.01386809836764842, RMSE = 0.09776325139686762
Epoch 800, a = 2.0305876697976712, b = 0.01985877283671398, RMSE = 0.10340348415216669
Epoch 900, a = 2.0071413894228693, b = 0.020092355664442216, RMSE = 0.0974972431814113
SGD method: a = 1.9950152479865568, b = 0.10762451212940533, RMSE = 0.0974972431814113, Time = 0.022143125534057617 seconds


In [6]:
# 시간 비교
print(f"SGD가, brute-force 방법에 비해 {brute_force_time / sgd_time:.2f}배 빠름")

SGD가, brute-force 방법에 비해 218.28배 빠름
